<a href="https://colab.research.google.com/github/daniel0076/EvaDBFinancialForecasting/blob/main/FinancialForecastingWithProfiling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CS6422 EvaDB Project 1: Financial Forecasting with Query Performance Evaluation

## Build an forecasting application with EvaDB

### Install dependencies

In [ ]:
!apt-get install -y postgresql
!service postgresql start

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libcommon-sense-perl libjson-perl libjson-xs-perl libtypes-serialiser-perl
  logrotate netbase postgresql-14 postgresql-client-14
  postgresql-client-common postgresql-common ssl-cert sysstat
Suggested packages:
  bsd-mailx | mailx postgresql-doc postgresql-doc-14 isag
The following NEW packages will be installed:
  libcommon-sense-perl libjson-perl libjson-xs-perl libtypes-serialiser-perl
  logrotate netbase postgresql postgresql-14 postgresql-client-14
  postgresql-client-common postgresql-common ssl-cert sysstat
0 upgraded, 13 newly installed, 0 to remove and 18 not upgraded.
Need to get 18.3 MB of archives.
After this operation, 51.5 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 logrotate amd64 3.19.0-1ubuntu1.1 [54.3 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main am

### Create database and install EvaDB

In [ ]:
!sudo -u postgres psql -c "CREATE USER eva WITH SUPERUSER PASSWORD 'password'"
!sudo -u postgres psql -c "CREATE DATABASE evadb"

CREATE ROLE
CREATE DATABASE


In [ ]:
#%pip install --quiet "evadb[postgres,forecasting] @ git+https://github.com/georgia-tech-db/evadb.git@master"
%pip install "evadb[postgres,forecasting] @ git+https://github.com/daniel0076/evadb.git@profiling"

  Cloning https://github.com/daniel0076/evadb.git (to revision profiling) to /tmp/pip-install-w23t77q1/evadb_49122d953e47448b901eab5acf2dc4e8
  Running command git clone --filter=blob:none --quiet https://github.com/daniel0076/evadb.git /tmp/pip-install-w23t77q1/evadb_49122d953e47448b901eab5acf2dc4e8
  Resolved https://github.com/daniel0076/evadb.git to commit c401b99d37cf28bb16b99442b1d41075ccf75946
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.9/108.9 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.9/110.9 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 kB

In [ ]:
import evadb
cursor = evadb.connect().cursor()
params = {
    "user": "eva",
    "password": "password",
    "host": "localhost",
    "port": "5432",
    "database": "evadb",
}
query = f"CREATE DATABASE postgres_data WITH ENGINE = 'postgres', PARAMETERS = {params};"
cursor.query(query).df()

Downloading: "http://ml.cs.tsinghua.edu.cn/~chenxi/pytorch-models/mnist-b07bb66b.pth" to /root/.cache/torch/hub/checkpoints/mnist-b07bb66b.pth
100%|██████████| 1.03M/1.03M [00:01<00:00, 1.04MB/s]
Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth


evadb.interfaces.relational.db.parse_query-start: 22:11:45.869937
evadb.interfaces.relational.db.parse_query-end:   22:11:45.875428, elapsed time(us): 0:00:00.005491
evadb.interfaces.relational.relation.execute-start: 22:11:45.875612
evadb.interfaces.relational.relation.execute-end:   22:11:45.965644, elapsed time(us): 0:00:00.090032


,0
0,The database postgres_data has been successful...


### Download the dateset and perform data cleaning

We need to

1. Remove all the null data
2. Remove duplicated rows

In [ ]:
!mkdir -p data
!wget -qnc -O data/sp500.zip https://github.com/CNuge/kaggle-code/raw/master/stock_data/individual_stocks_5yr.zip
!wget -qnc -O data/merge.sh https://github.com/CNuge/kaggle-code/raw/master/stock_data/merge.sh
!cd data && unzip sp500.zip
!cd data && sh merge.sh

Archive:  sp500.zip
   creating: individual_stocks_5yr/
  inflating: individual_stocks_5yr/XRX_data.csv  
  inflating: individual_stocks_5yr/GS_data.csv  
  inflating: individual_stocks_5yr/SPGI_data.csv  
  inflating: individual_stocks_5yr/MTB_data.csv  
  inflating: individual_stocks_5yr/V_data.csv  
  inflating: individual_stocks_5yr/CTAS_data.csv  
  inflating: individual_stocks_5yr/ESRX_data.csv  
  inflating: individual_stocks_5yr/APH_data.csv  
  inflating: individual_stocks_5yr/BBT_data.csv  
  inflating: individual_stocks_5yr/RHI_data.csv  
  inflating: individual_stocks_5yr/GGP_data.csv  
  inflating: individual_stocks_5yr/HCA_data.csv  
  inflating: individual_stocks_5yr/HD_data.csv  
  inflating: individual_stocks_5yr/PEP_data.csv  
  inflating: individual_stocks_5yr/INTC_data.csv  
  inflating: individual_stocks_5yr/MAC_data.csv  
  inflating: individual_stocks_5yr/CLX_data.csv  
  inflating: individual_stocks_5yr/UNP_data.csv  
  inflating: individual_stocks_5yr/WAT_data.

In [ ]:
import pandas as pd
df = pd.read_csv("/content/data/all_stocks_5yr.csv")
df.dropna(inplace=True) # To remove null data
df.drop_duplicates(inplace=True)
df.set_index('date', inplace=True) # Set the 'date' column as the index
df.to_csv("/content/data/stock_cleaned.csv")

### Create the table for the data in database

> Note that, when using forecasting with EvaDB based on [`statsforcast`](https://github.com/Nixtla/statsforecast), the `date` column need to be `VARCHAR` format rather than `DATE`, otherwise we got an error that `DATE` type is not supported in EvaDB

In [ ]:
cursor.query("""
  USE postgres_data {
    CREATE TABLE sp500 (
      date VARCHAR(64) NOT NULL,
      open NUMERIC(10, 2) NOT NULL,
      high NUMERIC(10, 2) NOT NULL,
      low NUMERIC(10, 2) NOT NULL,
      close NUMERIC(10, 2) NOT NULL,
      volume INT NOT NULL,
      name VARCHAR(255) NOT NULL
    )
  }
""").df()

evadb.interfaces.relational.db.parse_query-start: 22:11:55.859766
evadb.interfaces.relational.db.parse_query-end:   22:11:55.863147, elapsed time(us): 0:00:00.003381
evadb.interfaces.relational.relation.execute-start: 22:11:55.863301
evadb.interfaces.relational.relation.execute-end:   22:11:55.986769, elapsed time(us): 0:00:00.123468


,status
0,success


### Load the cleaned data from the CSV into database and EvaDB

In [ ]:
cursor.query("""
  USE postgres_data {
    COPY sp500(date, open, high, low, close, volume, name)
    FROM '/content/data/stock_cleaned.csv'
    DELIMITER ',' CSV HEADER
  }
""").df()

evadb.interfaces.relational.db.parse_query-start: 22:11:56.013665
evadb.interfaces.relational.db.parse_query-end:   22:11:56.016930, elapsed time(us): 0:00:00.003265
evadb.interfaces.relational.relation.execute-start: 22:11:56.017032
evadb.interfaces.relational.relation.execute-end:   22:11:58.705071, elapsed time(us): 0:00:02.688039


,status
0,success


> We can preview the data with SQL

In [ ]:
cursor.query("SELECT * FROM postgres_data.sp500 LIMIT 3;").df()

evadb.interfaces.relational.db.parse_query-start: 22:11:58.726577
evadb.interfaces.relational.db.parse_query-end:   22:11:58.745491, elapsed time(us): 0:00:00.018914
evadb.interfaces.relational.relation.execute-start: 22:11:58.746312


/usr/local/lib/python3.10/dist-packages/evadb/third_party/databases/postgres/postgres_handler.py:93: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  tables_df = pd.read_sql_query(query, self.connection)
/usr/local/lib/python3.10/dist-packages/evadb/third_party/databases/postgres/postgres_handler.py:111: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  columns_df = pd.read_sql_query(query, self.connection)
/usr/local/lib/python3.10/dist-packages/evadb/third_party/databases/postgres/postgres_handler.py:93: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please conside

evadb.interfaces.relational.relation.execute-end:   22:12:08.318755, elapsed time(us): 0:00:09.572443


,sp500.low,sp500.open,sp500.high,sp500.close,sp500.volume,sp500.date,sp500.name
0,14.63,15.07,15.12,14.75,8407500,2013-02-08,AAL
1,14.26,14.89,15.01,14.46,8882000,2013-02-11,AAL
2,14.10,14.45,14.51,14.27,8126000,2013-02-12,AAL


## Analysis Data with EvaDB, here we try to predict the closing price of each symbol

+ `PREDICT`: the column to predict
+ `TIME`: The column for time series data
+ `ID`: The identifier to group data (for multiple time series)

In [ ]:
cursor.query("DROP FUNCTION IF EXISTS stockForecast;").df()
cursor.query("""
  CREATE FUNCTION IF NOT EXISTS stockForecast FROM
    (
      SELECT name, date, close
      FROM postgres_data.sp500
    )
  TYPE Forecasting
  PREDICT 'close'
  TIME 'date'
  ID 'name'
  FREQUENCY 'D'
""").df()

10-10-2023 22:12:08 WARNING[drop_object_executor:drop_object_executor.py:_handle_drop_function:0086] Function stockForecast does not exist, therefore cannot be dropped.


evadb.interfaces.relational.db.parse_query-start: 22:12:08.341584
evadb.interfaces.relational.db.parse_query-end:   22:12:08.346392, elapsed time(us): 0:00:00.004808
evadb.interfaces.relational.relation.execute-start: 22:12:08.347048
evadb.interfaces.relational.relation.execute-end:   22:12:08.366534, elapsed time(us): 0:00:00.019486
evadb.interfaces.relational.db.parse_query-start: 22:12:08.367398
evadb.interfaces.relational.db.parse_query-end:   22:12:08.376292, elapsed time(us): 0:00:00.008894
evadb.interfaces.relational.relation.execute-start: 22:12:08.376869


/usr/local/lib/python3.10/dist-packages/evadb/third_party/databases/postgres/postgres_handler.py:93: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  tables_df = pd.read_sql_query(query, self.connection)
/usr/local/lib/python3.10/dist-packages/evadb/third_party/databases/postgres/postgres_handler.py:111: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  columns_df = pd.read_sql_query(query, self.connection)
/usr/local/lib/python3.10/dist-packages/evadb/third_party/databases/postgres/postgres_handler.py:93: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please conside

evadb.interfaces.relational.relation.execute-end:   22:15:48.615457, elapsed time(us): 0:03:40.238588


,0
0,Function stockForecast added to the database.


### Use the model to predict a symbol

For example we want to find the price of `NVDA` in the upcoming 50 days

In [ ]:
cursor.query("SELECT * FROM (SELECT stockForecast(50)) AS S WHERE name = 'NVDA' ORDER BY date ;").df()

evadb.interfaces.relational.db.parse_query-start: 22:15:48.632667
evadb.interfaces.relational.db.parse_query-end:   22:15:48.658528, elapsed time(us): 0:00:00.025861
evadb.interfaces.relational.relation.execute-start: 22:15:48.658995
evadb.interfaces.relational.relation.execute-end:   22:15:50.261256, elapsed time(us): 0:00:01.602261


,S.name,S.date,S.close
0,NVDA,2018-02-08,229.091492
1,NVDA,2018-02-09,229.518341
2,NVDA,2018-02-10,229.938919
3,NVDA,2018-02-11,230.359802
4,NVDA,2018-02-12,230.780670
5,NVDA,2018-02-13,231.201538
6,NVDA,2018-02-14,231.622406
7,NVDA,2018-02-15,232.043274
8,NVDA,2018-02-16,232.464127
9,NVDA,2018-02-17,232.884995
